## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import math
from scipy import stats as st
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [2]:
BUDGET = 10_000_000_000 # Общий бюджет на регион
COST_PER_1_PROD = 450_000 #Стоимость 1 единицы продукции
COUNT_POINT = 200 #Сколько скважин будет построено

In [3]:
def df_info(region,region_number): #Просмотр информации о датасетах
    print(region_number)
    print('---------')
    display(region.head())
    print('----------------------------------------------------------------------------')
    print(region.info())
    print('----------------------------------------------------------------------------')
    print(region.describe())
    
def model_learning(reg): #Функция для разделения данных и обучения модели
    features = reg.drop(['id','product'], axis=1) #Отделение признаков
    target = reg['product'] #Отделение целевого столбца
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target,test_size=0.25,random_state=5678) #Разделение выборки на тестовую и валидационную с пропорциями 75%:25%

    scaler=StandardScaler() #Масштабирование признаков
    scaler.fit(features_train)
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    model = LinearRegression()
    model.fit(features_train,target_train) #Обучение модели
    predictions = model.predict(features_valid) #предсказание модели на валидационной выборке
    
    print('Средний запас предсказанного сырья:',predictions.mean()) 
    print('Реальный средний запас:',target_valid.mean())
    mse = mean_squared_error(target_valid,predictions,squared=False) 
    print('RMSE модели =',mse)
    predictions_mean = pd.Series(target_valid.mean(),index=target_valid.index)
    mse = mean_squared_error(target_valid,predictions_mean,squared=False)
    print('RMSE случайный',mse)
    
    return predictions, target_valid

def income_counting(predictions,target): #Функция определения 200 лучших скважин и тогового дохода с них
    predictions.sort_values(inplace=True)
    predictions = predictions[-200:]
    income = target[predictions.index].sum()*COST_PER_1_PROD
    print('Итоговый доход с крупнейших скважин в регионе:',income) 
    if income < BUDGET:
        print('Доходы с крупнейших скважин будут меньше чем затраты')
    else:
        print('Доходы с крупнейших скважин будут больше чем затраты')
        
    print()
    
def income_counting_for_boot(predictions,target): #Аналогична функции сверху (использует в функции бутстреппа)
    predictions.sort_values(inplace=True)
    predictions = predictions[-200:]
    income = target[predictions.index].sum()*COST_PER_1_PROD
    return income
        
def bootstrapping(predictions,target, region): #Функция бутстреппа с подсчетом риска, среднего значения и 95% квантиля
    print(region)
    state = np.random.RandomState(3456)
    values = []
    for i in range(1000): #Бутстрепп
        subsample = predictions.sample(500,replace=True,random_state=state) #Берем случайные 500 точек
        values.append(income_counting_for_boot(subsample,target)) #Добавляем в список доходы с лучших 200 площадок 
    values = pd.Series(values)
    up_interval = values.quantile(0.975)
    low_interval = values.quantile(0.025)
    risk = 0
    for i in range(1000): #Подсчет риска
        if values[i]<BUDGET:
            risk+=1
    print('Количество рисковых объектов',risk)
    risk_percent = risk/1000*100
    print('Процент риска',risk_percent)
    print('Средняя выручка с региона:',values.mean())
    print('95% доверительный интервал',low_interval,' ',up_interval)
    if risk_percent>2.5:
        print('Вероятность убытков больше 2,5%')
    else:
        print('Вероятность убытков меньше 2,5%, можно разрабатывать месторождения в регионе')

In [4]:
reg_1 = pd.read_csv('/datasets/geo_data_0.csv')
reg_2 = pd.read_csv('/datasets/geo_data_1.csv')
reg_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [5]:
df_info(reg_1,'1 регион')

1 регион
---------


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


----------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
----------------------------------------------------------------------------
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%        

In [6]:
df_info(reg_2,'2 регион')

2 регион
---------


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


----------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
----------------------------------------------------------------------------
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%        

In [7]:
df_info(reg_3,'3 регион')

3 регион
---------


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


----------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
----------------------------------------------------------------------------
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.002023      -0.002081       2.495128      95.000000
std         1.732045       1.730417       3.473445      44.749921
min        -8.760004      -7.084020     -11.970335       0.000000
25%        -1.162288      -1.174820       0.130359      59.450441
50%        

Пропусков в данных не обнаружено, проверить данные на аномалии мы не можем, так как не знаем что именно за значения в столбца `f1`,`f2`,`f3`, единственное что мы можем проверить на неправильные значения это столбец `product`, в котором не может быть отрицательных значений, их там и не было обнаружено. Все данные представлены в правильном формате и в корректировке не нуждаются. По итогу можно сказать, что данные готовы к работе

## Обучение и проверка модели

In [8]:
reg_1=shuffle(reg_1) #Перемешиваем датафрейм
reg_2=shuffle(reg_2) #Перемешиваем датафрейм
reg_3=shuffle(reg_3) #Перемешиваем датафрейм

In [9]:
predictions_1, right_answers_1 = model_learning(reg_1)
predictions_1 = pd.Series(predictions_1,index=right_answers_1.index) 

Средний запас предсказанного сырья: 92.40785293185843
Реальный средний запас: 92.26607524822349
RMSE модели = 37.57307919019012
RMSE случайный 44.22529783010696


In [10]:
predictions_2, right_answers_2 = model_learning(reg_2)
predictions_2 = pd.Series(predictions_2,index=right_answers_2.index) 

Средний запас предсказанного сырья: 69.13663464963116
Реальный средний запас: 69.12766162366945
RMSE модели = 0.8961919269074418
RMSE случайный 46.04249297217585


In [11]:
predictions_3, right_answers_3 = model_learning(reg_3)
predictions_3 = pd.Series(predictions_3,index=right_answers_3.index) 

Средний запас предсказанного сырья: 95.14486775195927
Реальный средний запас: 94.72468627811361
RMSE модели = 40.11321176281278
RMSE случайный 44.845669980580496


По итогу обучения моделей и проверки на обучающих выборках мы можем увидеть, что модели неплохо справились с задачей (особенно модель во втором регионе, чем RMSE составил меньше 1), при это все модели показали значения RMSE ниже случайного, а средние значения предсказанного сырья во всех моделях не расходилось с настоязим средним более чем на 1

## Подготовка к расчёту прибыли

In [12]:
print('Достаточный объём сырья для безубыточной разработки новой скважины:',
      math.ceil(BUDGET/COUNT_POINT/COST_PER_1_PROD))

Достаточный объём сырья для безубыточной разработки новой скважины: 112


Во всех регионах в среднем запасы единиц продукции меньше чем достаточный объем сырья для безубыточной разработки новой скважины, в среднем больше всего единиц продукции находится в регионе 3, меньше всего в регионе 2. Для максимальной прибыли нам нужно выбрать наиболее богатые месторождения

In [13]:
income_counting(predictions_1,right_answers_1)
income_counting(predictions_2,right_answers_2)
income_counting(predictions_3,right_answers_3)

Итоговый доход с крупнейших скважин в регионе: 13284601704.7012
Доходы с крупнейших скважин будут больше чем затраты

Итоговый доход с крупнейших скважин в регионе: 12415086696.68151
Доходы с крупнейших скважин будут больше чем затраты

Итоговый доход с крупнейших скважин в регионе: 12198954155.273672
Доходы с крупнейших скважин будут больше чем затраты



После проведения рассчетов можно сказать что все регионы могут быть выгодными

## Расчёт прибыли и рисков 

In [14]:
bootstrapping(predictions_1,right_answers_1,'1 Регион')

1 Регион
Количество рисковых объектов 49
Процент риска 4.9
Средняя выручка с региона: 10438908034.607695
95% доверительный интервал 9928857477.070282   10950074023.00059
Вероятность убытков больше 2,5%


In [15]:
bootstrapping(predictions_2,right_answers_2,'2 Регион')

2 Регион
Количество рисковых объектов 7
Процент риска 0.7000000000000001
Средняя выручка с региона: 10484892704.203327
95% доверительный интервал 10075174068.468569   10891733478.057922
Вероятность убытков меньше 2,5%, можно разрабатывать месторождения в регионе


In [16]:
bootstrapping(predictions_3,right_answers_3,'3 Регион')

3 Регион
Количество рисковых объектов 83
Процент риска 8.3
Средняя выручка с региона: 10371106604.716072
95% доверительный интервал 9811273065.372887   10970340800.783724
Вероятность убытков больше 2,5%


По итогам расчета прибыли, если мы из регионов будем исследоваать 500 случайных точек и выбирать среди них 200 лучших, то наименее рискованным будет 2 регион, при этом и средняя прибыль с региона будет практически самая высокая, так что лучше всего заниматься разработкой месторождений именно во втором регионе